In [1]:
!pip install -U keras-tuner

                                              0.0/129.5 kB ? eta -:--:--
     ---------                               30.7/129.5 kB 1.4 MB/s eta 0:00:01
     -----------------                     61.4/129.5 kB 825.8 kB/s eta 0:00:01
     ------------------------------------ 129.5/129.5 kB 953.8 kB/s eta 0:00:00
                                              0.0/950.8 kB ? eta -:--:--
     --                                      51.2/950.8 kB 2.6 MB/s eta 0:00:01
     --                                      51.2/950.8 kB 2.6 MB/s eta 0:00:01
     ---                                   81.9/950.8 kB 651.6 kB/s eta 0:00:02
     -------                                194.6/950.8 kB 1.2 MB/s eta 0:00:01
     ------------                           317.4/950.8 kB 1.5 MB/s eta 0:00:01
     -------------                          348.2/950.8 kB 1.4 MB/s eta 0:00:01
     --------------------                   501.8/950.8 kB 1.7 MB/s eta 0:00:01
     ------------------------               614.4/950.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Dense
import keras_tuner as kf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

Using TensorFlow backend


In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/aishwaryamate/Datasets/main/Churn_Modelling.csv')

In [4]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
df.drop(columns = ['RowNumber'	,'CustomerId',	'Surname'], inplace=True)

In [6]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [7]:
df = pd.get_dummies(data = df,columns=['Geography','Gender'])
df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,1,0,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,1,0,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,1,1,0,0,1,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1,0,0,1


In [8]:
x = df.drop(columns = ['Exited'])
y = df['Exited']

In [9]:
sc = StandardScaler()

x = sc.fit_transform(x)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size = 0.20, random_state=1)

In [12]:
ann = Sequential()

ann.add(Dense(units = 10, activation = 'relu'))
ann.add(Dense(units = 1, activation = 'sigmoid'))

ann.compile(optimizer = 'adadelta', loss = 'binary_crossentropy', metrics ='accuracy')

ann.fit(xtrain,ytrain,epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.8377 - accuracy: 0.4121
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.8363 - accuracy: 0.4124
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.8348 - accuracy: 0.4134
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.8333 - accuracy: 0.4144
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.8318 - accuracy: 0.4155
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.8302 - accuracy: 0.4165
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.8286 - accuracy: 0.4174
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.8271 - accuracy: 0.4181
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.8255 - accuracy: 0.4195
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.8238 - accura

250/250 [==============================] - 1s 3ms/step - loss: 0.7231 - accuracy: 0.5042
Epoch 82/100
250/250 [==============================] - 1s 3ms/step - loss: 0.7219 - accuracy: 0.5055
Epoch 83/100
250/250 [==============================] - 1s 2ms/step - loss: 0.7207 - accuracy: 0.5064
Epoch 84/100
250/250 [==============================] - 0s 2ms/step - loss: 0.7195 - accuracy: 0.5073
Epoch 85/100
250/250 [==============================] - 0s 2ms/step - loss: 0.7183 - accuracy: 0.5079
Epoch 86/100
250/250 [==============================] - 0s 1ms/step - loss: 0.7172 - accuracy: 0.5095
Epoch 87/100
250/250 [==============================] - 0s 1ms/step - loss: 0.7160 - accuracy: 0.5109
Epoch 88/100
250/250 [==============================] - 0s 1ms/step - loss: 0.7148 - accuracy: 0.5117
Epoch 89/100
250/250 [==============================] - 0s 2ms/step - loss: 0.7137 - accuracy: 0.5125
Epoch 90/100
250/250 [==============================] - 0s 2ms/step - loss: 0.7125 - accuracy: 

In [13]:
ypred = ann.predict(xtest)
ypred = ypred > 0.5
ypred

63/63 [==============================] - 0s 2ms/step


array([[ True],
       [ True],
       [ True],
       ...,
       [False],
       [ True],
       [False]])

In [14]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.77      0.58      0.66      1585
           1       0.18      0.35      0.24       415

    accuracy                           0.53      2000
   macro avg       0.48      0.47      0.45      2000
weighted avg       0.65      0.53      0.58      2000



In [15]:
ann.evaluate(xtrain,ytrain)

250/250 [==============================] - 1s 2ms/step - loss: 0.7008 - accuracy: 0.5274


[0.7008104920387268, 0.5273749828338623]

# Hyperparameter tuning


In [31]:
counter = 0

for i in range(0,11):
  if counter == 0:
    print('Python')
else:
    print('Machine learning')
counter = counter+1

Python
Python
Python
Python
Python
Python
Python
Python
Python
Python
Python
Machine learning


In [17]:
counter

1

In [18]:
def hyper(hp):
    model = Sequential()
    counter = 0
    for i in range(hp.Int('num_layers', min_value = 1,max_value=10)):
        if counter ==0:
            model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                            activation = hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid']),
                            input_dim = 13))
    else:
        model.add(Dense(
              hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
              activation = hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid'])))
    counter+=1
    model.add(Dense(units = 1,activation='sigmoid'))
    model.compile(optimizer = hp.Choice('optimizer',values = ['adam','rmsprop','sgd']),
                loss = 'binary_crossentropy',metrics = 'accuracy')
    return model

In [19]:
tuner = kf.RandomSearch(hyper,
                        objective = 'val_accuracy',
                        max_trials = 3)

In [20]:
tuner.search(xtrain,ytrain,epochs = 5, validation_data = (xtest,ytest))

Trial 3 Complete [00h 00m 06s]
val_accuracy: 0.7925000190734863

Best val_accuracy So Far: 0.8575000166893005
Total elapsed time: 00h 00m 19s


In [21]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6,
 'units0': 48,
 'activation0': 'tanh',
 'optimizer': 'adam',
 'units1': 24,
 'activation1': 'relu',
 'units2': 88,
 'activation2': 'sigmoid',
 'units3': 24,
 'activation3': 'sigmoid',
 'units4': 72,
 'activation4': 'sigmoid',
 'units5': 88,
 'activation5': 'sigmoid',
 'units6': 80,
 'activation6': 'tanh',
 'units7': 96,
 'activation7': 'sigmoid',
 'units8': 48,
 'activation8': 'relu'}

In [22]:
df.shape

(10000, 14)

In [23]:
tuned_model = tuner.get_best_models(num_models=1)[0]
tuned_model.build(x.shape)

In [24]:
tuned_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 48)                672       
                                                                 
 dense_1 (Dense)             (None, 24)                1176      
                                                                 
 dense_2 (Dense)             (None, 88)                2200      
                                                                 
 dense_3 (Dense)             (None, 24)                2136      
                                                                 
 dense_4 (Dense)             (None, 72)                1800      
                                                                 
 dense_5 (Dense)             (None, 88)                6424      
                                                                 
 dense_6 (Dense)             (None, 88)                7

In [25]:
tuned_model.fit(xtrain,ytrain,epochs=50, validation_data=(xtest,ytest))

Epoch 1/50
250/250 [==============================] - 3s 4ms/step - loss: 0.3481 - accuracy: 0.8581 - val_loss: 0.3773 - val_accuracy: 0.8540
Epoch 2/50
250/250 [==============================] - 1s 2ms/step - loss: 0.3462 - accuracy: 0.8575 - val_loss: 0.3424 - val_accuracy: 0.8630
Epoch 3/50
250/250 [==============================] - 1s 2ms/step - loss: 0.3427 - accuracy: 0.8587 - val_loss: 0.3530 - val_accuracy: 0.8550
Epoch 4/50
250/250 [==============================] - 1s 2ms/step - loss: 0.3414 - accuracy: 0.8604 - val_loss: 0.3403 - val_accuracy: 0.8595
Epoch 5/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3381 - accuracy: 0.8605 - val_loss: 0.3425 - val_accuracy: 0.8605
Epoch 6/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3381 - accuracy: 0.8601 - val_loss: 0.3386 - val_accuracy: 0.8635
Epoch 7/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3344 - accuracy: 0.8626 - val_loss: 0.3372 - val_accuracy: 0.8625
Epoch 

In [26]:
tuned_model.evaluate(xtrain,ytrain)

250/250 [==============================] - 1s 2ms/step - loss: 0.2951 - accuracy: 0.8767


[0.29505211114883423, 0.8767499923706055]

In [27]:
tuned_model.evaluate(xtest,ytest)

63/63 [==============================] - 0s 3ms/step - loss: 0.3727 - accuracy: 0.8505


[0.37267452478408813, 0.8504999876022339]